In [1]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (Model, RealParameter, Constant, CategoricalParameter, 
                           TimeSeriesOutcome, ScalarOutcome, ema_logging, perform_experiments, 
                           MultiprocessingEvaluator, Policy)
from ema_workbench.connectors.vensim import (VensimModel , VensimModelStructureInterface, set_value)
from ema_workbench.em_framework.samplers import sample_levers
from ema_workbench.util.utilities import save_results
from ema_workbench import load_results
from ema_workbench.analysis import pairs_plotting
from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter)
from ema_workbench.analysis import clusterer, plotting, Density, prim
from ema_workbench.analysis.plotting import lines, envelopes, kde_over_time, multiple_densities



import timeit
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from ema_workbench.analysis.plotting import lines

C:\Programs\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath

# Select vensim file

In [2]:
wd = r'./VENSIM' # Directory of the vensim model
model_file=r'Maasmodel_v075.vpmx' # Name of the vensim model

# General manual input

In [3]:
#FILL IN THIS CELL
#WEIR
Crest_level_weir1 = 8.35
Crest_level_weir2 = 7.15
Crest_level_weir3 = 6.9
Crest_level_weir4 = 7.2
Crest_level_weir5 = 8.4
Crest_level_weir6 = 10.9
Location_weir1    = 57500
Location_weir2    = 69500
Location_weir3    = 87500
Location_weir4    = 135500
Location_weir5    = 163500
Location_weir6    = 189500
Leakage_low_weir1    = 1.2e6
Leakage_low_weir2    = 1e6
Leakage_low_weir3    = 1e6
Leakage_low_weir4    = 3e6
Leakage_low_weir5    = 3e6
Leakage_low_weir6    = 2e6
Leakage_up_weir1     = 6e6
Leakage_up_weir2     = 5e6
Leakage_up_weir3     = 5e6
Leakage_up_weir4     = 9e6
Leakage_up_weir5     = 9e6
Leakage_up_weir6     = 8e6

#LOCK
Intensity_commercial_lock1 = 44
Intensity_commercial_lock2 = 62
Intensity_commercial_lock3 = 439
Intensity_commercial_lock4 = 547
Intensity_commercial_lock5 = 289
Intensity_commercial_lock6 = 334
Intensity_leisure_lock1 = 643
Intensity_leisure_lock2 = 518
Intensity_leisure_lock3 = 458
Intensity_leisure_lock4 = 498
Intensity_leisure_lock5 = 511
Intensity_leisure_lock6 = 807

In [4]:
#NAME CLIMATE SCENARIOS
CS1 = 'Climate scenario GL'
CS2 = 'Climate scenario WH'

#SAVE FIGURE
save = False
fig_size = (10,7)
dpi = 300
%matplotlib inline

#Starting year
start_year = 2020
end_year   = 2100


#100 years
years = np.linspace(start_year,end_year,81)

#SECONDS TO Week
SEC_WEEK = 60*60*24*7 

#Different draught durations in weeks
Draught_durations = [4,6,8,10,12,14,16] #Duur van de droogte periode variërend van 4 tot 16 weken

#Different draught intensities in m3/s
Draught_intensities = [20,25,30,35,40,45,1000] #Maximale afvoer in periodes van droogte, 1000 staat voor geen droogte periode 

#Distance from inflow
distance_inflow = np.linspace(500, 189500, 190)

#100 year discharge series
year_100_dischargeseries = list(range(1911,2015))

# Model parameters

In [5]:
# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

# instantiate a model
vensimModel = VensimModel('MeuseModel', wd=wd, model_file=model_file)


# Experiment info and load the results

In [6]:
nr_scenarios = 8
nr_policies = 0    

policies = sample_levers([vensimModel], nr_policies)
policies = [policy for policy in policies]

print('Simulating the model with', nr_scenarios, 'scenarios and', nr_policies, 'design decisions')


Simulating the model with 8 scenarios and 0 design decisions


Load results

In [7]:
data = r'C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\BaseEnsemble_8000c.tar'
results = load_results(data)
experiments, outcomes = results

[MainProcess/INFO] results loaded succesfully from C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\BaseEnsemble_8000c.tar


# Data Postprocessing

In [8]:
keys = list(outcomes.keys())
keys

['TIME',
 'Percentage of full fresh water buffer[Weir1]',
 'Percentage of full fresh water buffer[Weir2]',
 'Percentage of full fresh water buffer[Weir3]',
 'Percentage of full fresh water buffer[Weir4]',
 'Percentage of full fresh water buffer[Weir5]',
 'Percentage of full fresh water buffer[Weir6]',
 'Average velocity through weir section[Weir1]',
 'Average velocity through weir section[Weir2]',
 'Average velocity through weir section[Weir3]',
 'Average velocity through weir section[Weir4]',
 'Average velocity through weir section[Weir5]',
 'Average velocity through weir section[Weir6]',
 '"I/C value at lock"[Weir1]',
 '"I/C value at lock"[Weir2]',
 '"I/C value at lock"[Weir3]',
 '"I/C value at lock"[Weir4]',
 '"I/C value at lock"[Weir5]',
 '"I/C value at lock"[Weir6]',
 'Days of free flow']

In [9]:
#Make dataframe with outcomes
fwb = ['Percentage of full fresh water buffer[Weir1]',
 'Percentage of full fresh water buffer[Weir2]',
 'Percentage of full fresh water buffer[Weir3]',
 'Percentage of full fresh water buffer[Weir4]',
 'Percentage of full fresh water buffer[Weir5]',
 'Percentage of full fresh water buffer[Weir6]','Days of free flow']
vel = keys[7:13]
ship= keys[13:19]


df_outcome = pd.DataFrame([[51]]*len(outcomes['TIME']))        
        
for j in range(len(fwb)):
    array = []
    for i in range(len(outcomes['TIME'])): 
        array.append(outcomes[fwb[j]][i][-1])
    df_outcome[fwb[j]] = array

for k in range(len(vel)):
    array1 = []
    for l in range(len(outcomes['TIME'])): 
        array1.append(np.mean(outcomes[vel[k]][l]))    
    df_outcome[vel[k]] = array1

for m in range(len(ship)):
    array2 = []
    for n in range(len(outcomes['TIME'])): 
        array2.append(np.max(outcomes[ship[m]][n]))    
    df_outcome[ship[m]] = array2  

del df_outcome[0]

In [10]:
#Add uncertainties to outcomes dataframe 
uncertain = experiments.keys().tolist()

for i in range(len(uncertain)):
    if experiments.keys()[i]==uncertain[i]:
        a = experiments.iloc[:,i].tolist()
        df_outcome[uncertain[i]] = a

In [11]:
#Give scenarios and policies a clear name
experiments[['Climate scenario']] = experiments[['Climate scenario']].replace([1, 2], 
                                               [CS1, CS2])


In [12]:
#Group simulation year

year_2020_2030 = np.linspace(2020,2029,10)
year_2030_2040 = np.linspace(2030,2039,10)
year_2040_2050 = np.linspace(2040,2049,10)
year_2050_2060 = np.linspace(2050,2059,10)
year_2060_2070 = np.linspace(2060,2069,10)
year_2070_2080 = np.linspace(2070,2079,10)
year_2080_2090 = np.linspace(2080,2089,10)
year_2090_2100 = np.linspace(2090,2100,11)
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2020_2030, 
                                               '2020-2030')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2030_2040, 
                                               '2030-2040')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2040_2050, 
                                               '2040-2050')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2050_2060, 
                                               '2050-2060')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2060_2070, 
                                               '2060-2070')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2070_2080, 
                                               '2070-2080')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2080_2090, 
                                               '2080-2090')
experiments[['Simulated year']] = experiments[['Simulated year']].replace(year_2090_2100, 
                                               '2090-2100')

In [13]:
#Group draught period intensity

discharge_20_25 = [20,21,22,23,24]
discharge_25_30 = [25,26,27,28,29]
discharge_30_35 = [30,31,32,33,34]
discharge_35_40 = [35,36,37,38,39]
discharge_40_45 = [40,41,42,43,44,45]

experiments[['Draught period intensity']] = experiments[['Draught period intensity']].replace(discharge_20_25, 
                                               '20-25 $m^3/s$')
experiments[['Draught period intensity']] = experiments[['Draught period intensity']].replace(discharge_25_30, 
                                               '25-30 $m^3/s$')
experiments[['Draught period intensity']] = experiments[['Draught period intensity']].replace(discharge_30_35, 
                                               '30-35 $m^3/s$')
experiments[['Draught period intensity']] = experiments[['Draught period intensity']].replace(discharge_35_40, 
                                               '35-40 $m^3/s$')
experiments[['Draught period intensity']] = experiments[['Draught period intensity']].replace(discharge_40_45, 
                                               '40-45 $m^3/s$')


In [15]:
#Save dataframe to excel 
df_outcome.to_excel(r"C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\base_dataframe.xlsx")